# Load Cora Dataset 

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
device = torch.device('cpu')
import numpy as np
import random

data = torch.load('data.pth')
g = data['g'].to(device)
feat = data['feat'].to(device)
label = data['label'].to(device)
train_nodes = data['train_nodes']
val_nodes = data['val_nodes']
test_nodes = data['test_nodes']

def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
setup_seed(20)

Using backend: pytorch


# Median GCN

*Chen et al.* [📝Understanding Structural Vulnerability in Graph Convolutional Networks](https://www.ijcai.org/proceedings/2021/310), *IJCAI'21*

In [2]:
import dgl
import dgl.function as fn
import dgl.ops as ops
from dgl import DGLError

def dgl_normalize(g: dgl.DGLGraph, norm: str = 'both', edge_weight=None):
    e_norm = torch.ones(g.num_edges(), device=g.device) if edge_weight is None else edge_weight

    if norm == 'none':
        return e_norm

    if edge_weight is None:
        src_degrees = g.in_degrees().clamp(min=1)
        dst_degrees = g.out_degrees().clamp(min=1)
    else:
        # a weighted graph
        src_degrees = dst_degrees = ops.copy_e_sum(g, edge_weight)
    if norm == 'left':
        # A * D^-1
        norm_src = 1.0 / src_degrees
        e_norm = ops.e_mul_v(g, e_norm, norm_src)
    elif norm == 'right':
        # D^-1 * A
        norm_dst = 1.0 / dst_degrees
        e_norm = ops.e_mul_u(g, e_norm, norm_dst)
    else:  # both or square
        if norm == 'both':
            # D^-0.5 * A * D^-0.5
            pow = -0.5
        else:
            # D^-1 * A * D^-1
            pow = -1
        norm_src = torch.pow(src_degrees, pow)
        norm_dst = torch.pow(dst_degrees, pow)
        e_norm = ops.e_mul_u(g, e_norm, norm_src)
        e_norm = ops.e_mul_v(g, e_norm, norm_dst)
    return e_norm

class MedianConv(nn.Module):
    def __init__(self,
                 in_feats,
                 out_feats,
                 norm='none',
                 activation=None,
                 weight=True,
                 bias=True):

        super().__init__()
        if norm not in ('none', 'both', 'right', 'left'):
            raise DGLError('Invalid norm value. Must be either "none", "both", "right" or "left".'
                           ' But got "{}".'.format(norm))
        self._in_feats = in_feats
        self._out_feats = out_feats
        self._norm = norm
        self._activation = activation

        if weight:
            self.weight = nn.Parameter(torch.Tensor(in_feats, out_feats))
        else:
            self.register_parameter('weight', None)

        if bias:
            self.bias = nn.Parameter(torch.Tensor(out_feats))
        else:
            self.register_parameter('bias', None)

        self.reset_parameters()

    def reset_parameters(self):
        r"""
        Description
        -----------
        Reinitialize learnable parameters.
        Note
        ----
        The model parameters are initialized as in the
        `original implementation <https://github.com/tkipf/gcn/blob/master/gcn/layers.py>`__
        where the weight :math:`W^{(l)}` is initialized using Glorot uniform initialization
        and the bias is initialized to be zero.
        """
        if self.weight is not None:
            nn.init.xavier_uniform_(self.weight)

        if self.bias is not None:
            nn.init.zeros_(self.bias)

    def forward(self, graph, feat):
        graph = graph.local_var()

        edge_weight = dgl_normalize(graph, self._norm)
        graph.edata['_edge_weight'] = edge_weight

        if self.weight is not None:
            feat = feat @ self.weight

        graph.ndata['h'] = feat
        graph.update_all(fn.u_mul_e('h', '_edge_weight', 'm'),
                         median_reduce)
        feat = graph.ndata.pop('h')

        if self.bias is not None:
            feat = feat + self.bias

        if self._activation is not None:
            feat = self._activation(feat)
        return feat

    def extra_repr(self):
        """Set the extra representation of the module,
        which will come into effect when printing the model.
        """
        summary = 'in={_in_feats}, out={_out_feats}'
        summary += ', normalization={_norm}'
        if '_activation' in self.__dict__:
            summary += ', activation={_activation}'
        return summary.format(**self.__dict__)


def median_reduce(nodes):
    return {'h': torch.median(nodes.mailbox['m'], dim=1).values}


class MedianGCN(nn.Module):
    """Graph Convolution Network (GCN) with Median aggregation function

    Example
    -------
    # MedianGCN with one hidden layer
    >>> model = MedianGCN(100, 10, hid=32)
    """
    def __init__(self,
                 in_feats: int,
                 out_feats: int,
                 hid: list = 16,
                 dropout: float = 0.5):
        super().__init__()
        self.conv1 = MedianConv(in_feats, hid)
        self.conv2 = MedianConv(hid, out_feats)
        self.dropout = nn.Dropout(dropout)

    def forward(self, g, feat):

        if torch.is_tensor(g):
            feat = self.dropout(feat)
            feat = g @ (feat @ self.conv1.weight) + self.conv1.bias
            feat = F.relu(feat)
            feat = self.dropout(feat)
            feat = g @ (feat @ self.conv2.weight) + self.conv2.bias
            return feat
        
        g = g.add_self_loop()
            
        feat = self.dropout(feat)
        feat = self.conv1(g, feat)
        feat = F.relu(feat)
        feat = self.dropout(feat)
        feat = self.conv2(g, feat)
        return feat


In [3]:
def train():
    model.train()
    optimizer.zero_grad()
    loss_fn(model(g, feat)[train_nodes], label[train_nodes]).backward()
    optimizer.step()


@torch.no_grad()
def test():
    model.eval()
    logits, accs = model(g, feat), []
    for nodes in (train_nodes, val_nodes, test_nodes):
        pred = logits[nodes].max(1)[1]
        acc = pred.eq(label[nodes]).float().mean()
        accs.append(acc)
    return accs

device = torch.device('cpu')
g = g.to(device)
feat = feat.to(device)
label = label.to(device)

num_feats = feat.size(1)
num_classes = int(label.max() + 1)
model = MedianGCN(num_feats, num_classes).to(device)

optimizer = torch.optim.Adam([
    dict(params=model.conv1.parameters(), weight_decay=5e-4),
    dict(params=model.conv2.parameters(), weight_decay=0)
], lr=0.01)  # Only perform weight-decay on first convolution.

loss_fn = nn.CrossEntropyLoss()

best_val_acc = test_acc = 0
for epoch in range(1, 101):
    train()
    train_acc, val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    print(f'Epoch: {epoch:03d}, Train: {train_acc:.4f}, '
          f'Val: {best_val_acc:.4f}, Test: {test_acc:.4f}')

Epoch: 001, Train: 0.2702, Val: 0.2369, Test: 0.2399
Epoch: 002, Train: 0.4194, Val: 0.3936, Test: 0.3959
Epoch: 003, Train: 0.5040, Val: 0.4498, Test: 0.4482
Epoch: 004, Train: 0.5282, Val: 0.4578, Test: 0.4658
Epoch: 005, Train: 0.5242, Val: 0.4819, Test: 0.4668
Epoch: 006, Train: 0.5040, Val: 0.4900, Test: 0.4673
Epoch: 007, Train: 0.5081, Val: 0.4900, Test: 0.4673
Epoch: 008, Train: 0.5161, Val: 0.4900, Test: 0.4673
Epoch: 009, Train: 0.5121, Val: 0.4900, Test: 0.4673
Epoch: 010, Train: 0.5282, Val: 0.4900, Test: 0.4673
Epoch: 011, Train: 0.5605, Val: 0.4900, Test: 0.4673
Epoch: 012, Train: 0.5766, Val: 0.4980, Test: 0.4834
Epoch: 013, Train: 0.6008, Val: 0.5181, Test: 0.5025
Epoch: 014, Train: 0.6452, Val: 0.5582, Test: 0.5272
Epoch: 015, Train: 0.6694, Val: 0.5783, Test: 0.5558
Epoch: 016, Train: 0.7056, Val: 0.6225, Test: 0.5951
Epoch: 017, Train: 0.7339, Val: 0.6827, Test: 0.6242
Epoch: 018, Train: 0.7460, Val: 0.7108, Test: 0.6449
Epoch: 019, Train: 0.7581, Val: 0.7269, Test: 

# Evaluation

The model can be evaluated using attacked graph `attack_g` generated by different attacks. 
Just using
```python

prediction = model(attack_g, feat)

```

In [4]:
d = torch.load('attack_graph.pth')
attack_g = d['attack_g']
attack_feat = d['attack_feat']
target = 1
model.eval()

MedianGCN(
  (conv1): MedianConv(in=1433, out=16, normalization=none, activation=None)
  (conv2): MedianConv(in=16, out=7, normalization=none, activation=None)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [5]:
model(g, feat)[target].argmax()

tensor(2)

In [6]:
# target gets correctly classified
model(attack_g, attack_feat)[target].argmax()

tensor(2)